# Load data

In [ ]:
import csv
import numpy as np

X = []
y = []
metadata = []

with open('primary_dataset.csv', 'r') as f:
    data = list(csv.reader(f))

header = data[0][6:36]
for line in data[1:]:
    scorer, video, diagnosis, age, sex = line[2], line[3], line[36], line[37], line[38]
    qs = line[6:36]
        
    # Remove scores with missing questions
    if len([x for x in qs if x == '']) == 0:
        X.append([int(x) for x in qs])
        y.append(int(diagnosis == 'asd'))
        metadata.append([scorer, video, age, sex])

X = np.asarray(X)
y = np.asarray(y)
m, n = X.shape

print('X', X.shape)
print('y', y.shape)
print('videos', len(set([x[1] for x in metadata])))
print('scorers', len(set([x[0] for x in metadata])))
print('% missing', np.sum(X>=7)/(X.shape[0]*X.shape[1]))

In [ ]:
# one hot encode responses
X_one_hot = []
header_one_hot = []
q_to_start_index = []
for q in range(30):
    q_to_start_index.append(len(header_one_hot))
    for response in np.unique(X[:, q]):
        if response < 7:
            X_one_hot.append(list(X[:, q]==response))
            header_one_hot.append((header[q], response))
q_to_start_index.append(len(header_one_hot))
X_one_hot = np.asarray(X_one_hot).T
print('X one hot', X_one_hot.shape)
print('X one hot header', len(header_one_hot))
print('q_to_start_index', q_to_start_index)

In [ ]:
# independent test data
Xind = []
yind = []
metadataind = []

with open('microbiome_updated.csv', 'r') as f:
    data = list(csv.reader(f))

for line in data[1:]:
    scorer, video, diagnosis, age, sex = line[2], line[3], line[36], line[37], line[38]
    qs = line[6:36]
        
    # Remove scores with missing questions
    if len([x for x in qs if x == '']) == 0:
        Xind.append([int(x) for x in qs])
        yind.append(int(diagnosis == 'asd'))
        metadataind.append([scorer, video, age, sex])

Xind = np.asarray(Xind)
yind = np.asarray(yind)
print(Xind.shape, yind.shape)
print('scorers', len(set([x[0] for x in metadataind])))
print('videos', len(set([x[1] for x in metadataind])))
print('% missing', np.sum(Xind>=7)/(Xind.shape[0]*Xind.shape[1]))

# one hot encode responses
Xind_one_hot = []
for q in range(30):
    for response in np.unique(X[:, q]):
        if response < 7:
            Xind_one_hot.append(list(Xind[:, q]==response))
Xind_one_hot = np.asarray(Xind_one_hot).T
print(Xind_one_hot.shape)

In [ ]:
# median impute missing values in raw data
meds = np.asarray([np.median(z[z<7]) for z in X.T])
missing_indices = np.where(X>=7)
print(X[missing_indices])
X[missing_indices] = meds[missing_indices[1]]
print(X[missing_indices])

# Split train/test
We tried splitting by video and also by rater. At first, we thought correlation might exists between two different raters rating the same video. However, after some experimentation we noticed that each rater seems to have his/her own technique and we were overfitting to rater more than to video. 

In [ ]:
import random
import math
from itertools import chain

def split_by_video(videos, k):
    shuffled_videos = list(videos)
    random.shuffle(shuffled_videos)
    num_videos_per_group = int(math.floor(len(shuffled_videos)/k))
    split = []
    videos = []
    for i in range(k):
        split_videos = set(shuffled_videos[(num_videos_per_group*i):(num_videos_per_group*(i+1))])
        split.append(list(np.where([x[1] in split_videos for x in metadata])[0]))
        videos.append(split_videos)
    return split, videos

def split_by_rater(raters, k):
    shuffled_raters = list(raters)
    random.shuffle(shuffled_raters)
    num_raters_per_group = int(math.floor(len(shuffled_raters)/k))
    split = []
    raters = []
    for i in range(k):
        split_raters = set(shuffled_raters[(num_raters_per_group*i):(num_raters_per_group*(i+1))])
        split.append(list(np.where([x[0] in split_raters for x in metadata])[0]))
        raters.append(split_raters)
    return split, raters


unique_videos = set([x[1] for x in metadata])
print('unique videos', len(unique_videos))

unique_raters = set([x[0] for x in metadata])
print('unique raters', len(unique_raters))

split, videos = split_by_video(unique_videos, 5)
test_indices, test_videos = np.asarray(split[0]), videos[0]
train_indices, train_videos = np.asarray(list(chain.from_iterable(split[1:5]))), set(chain.from_iterable(videos[1:5]))

#split, raters = split_by_rater(unique_raters, 3)
#test_indices, test_raters = np.asarray(split[0]), raters[0]
#train_indices, train_raters = np.asarray(list(chain.from_iterable(split[1:3]))), set(chain.from_iterable(raters[1:3]))

print('train_videos', len(train_videos))
print('test_videos', len(test_videos))
#print('train raters', len(train_raters))
#print('test raters', len(test_raters))
print(train_indices.shape, test_indices.shape)

# Basic L1 and L2 Models
Take a quick look at performance of L2 vs L1 regularization. However, we need to take these results with a grain of salt since I haven't tuned any of the parameters.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_curve, auc

def print_stats(model, X_train, y_train, X_test, y_test):
    # Nonzero entries
    print('nonzero coefficients:', np.sum(model.coef_[0]!=0), 'out of', model.coef_.shape[1])

    # Accuracy
    print('train accuracy:', sgdc.score(X_train, y_train))
    print('test accuracy:', sgdc.score(X_test, y_test))
    
    # AUC
    yhat = sgdc.predict_proba(X_train)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_train, yhat)
    print('train auc:', auc(fpr, tpr))
    yhat = sgdc.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, yhat)
    print('test auc:', auc(fpr, tpr))

# ---------------------------- L2 ----------------------------
print('L2 Raw Data')
sgdc = SGDClassifier(loss="log", penalty='l2', alpha=0.1, fit_intercept=True)
sgdc.fit(X[train_indices, :], y[train_indices])
print_stats(sgdc, X[train_indices, :], y[train_indices], X[test_indices, :], y[test_indices])

print('\nL2 One-hot Data')
sgdc = SGDClassifier(loss="log", penalty='l2', alpha=0.1, fit_intercept=True)
sgdc.fit(X_one_hot[train_indices, :], y[train_indices])
print_stats(sgdc, X_one_hot[train_indices, :], y[train_indices], X_one_hot[test_indices, :], y[test_indices])

# ---------------------------- L1 ----------------------------
print('\nL1 Raw Data')
sgdc = SGDClassifier(loss="log", penalty='l1', alpha=0.05, fit_intercept=True)
sgdc.fit(X[train_indices, :], y[train_indices])
print_stats(sgdc, X[train_indices, :], y[train_indices], X[test_indices, :], y[test_indices])

print('\nL1 One-hot Data')
sgdc = SGDClassifier(loss="log", penalty='l1', alpha=0.05, fit_intercept=True)
sgdc.fit(X_one_hot[train_indices, :], y[train_indices])
print_stats(sgdc, X_one_hot[train_indices, :], y[train_indices], X_one_hot[test_indices, :], y[test_indices])


First, we notice that as expected, that L2 outperforms L1 (it gets to use many more features to make predictions). However, even with a small number of features, and no parameter tuning, the L1 models are doing pretty well. This is promising. Comparing raw data to one-hot encoded data, it looks like the one-hot encoding may be performing better, but the numbers are pretty close.

# Tune an L2 Model
We start by tuning an L2 model. This let's us know the best possible performance we can expect from this data.

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as st

def tune_L2(X, k, alphas):
    accuracy_avg = []
    accuracy_err = []
    auc_avg = []
    auc_err = []

    for alpha in alphas:
        accuracy = np.zeros((k,))
        auroc = np.zeros((k,))

        for i, (split_test_indices, split_test_videos) in enumerate(zip(*split_by_video(train_videos, k))):
            split_train_indices = list(set(train_indices) - set(split_test_indices))

            # fit model
            sgdc = SGDClassifier(loss="log", penalty='l2', alpha=alpha, fit_intercept=True)
            sgdc.fit(X[split_train_indices, :], y[split_train_indices])

            # accuracy
            accuracy[i] = sgdc.score(X[split_test_indices, :], y[split_test_indices])

            # auc
            yhat = sgdc.predict_proba(X[split_test_indices, :])[:, 1]
            fpr, tpr, thresholds = roc_curve(y[split_test_indices], yhat)
            auroc[i] = auc(fpr, tpr)

        # calculate mean and 95% confidence intervals
        accuracy_avg.append(np.mean(accuracy))
        ci = st.t.interval(0.95, k-1, loc=np.mean(accuracy), scale=st.sem(accuracy))
        accuracy_err.append((ci[1]-ci[0])/2)
        auc_avg.append(np.mean(auroc))
        ci = st.t.interval(0.95, k-1, loc=np.mean(auroc), scale=st.sem(auroc))
        auc_err.append((ci[1]-ci[0])/2)
    return accuracy_avg, accuracy_err, auc_avg, auc_err
    

k = 5
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
accuracy_avg_raw, accuracy_err_raw, auc_avg_raw, auc_err_raw = tune_L2(X, k, alphas)
accuracy_avg_onehot, accuracy_err_onehot, auc_avg_onehot, auc_err_onehot = tune_L2(X_one_hot, k, alphas)

plt.figure(figsize=(10, 10))

# Accuracy
ax1 = plt.subplot(2, 1, 1)

plt.plot(alphas, accuracy_avg_raw, label='raw data')
plt.fill_between(alphas, [x+e for x, e in zip(accuracy_avg_raw, accuracy_err_raw)], [x-e for x, e in zip(accuracy_avg_raw, accuracy_err_raw)], alpha=0.5)

plt.plot(alphas, accuracy_avg_onehot, label='one-hot data')
plt.fill_between(alphas, [x+e for x, e in zip(accuracy_avg_onehot, accuracy_err_onehot)], [x-e for x, e in zip(accuracy_avg_onehot, accuracy_err_onehot)], alpha=0.5)

plt.xlabel('Regularization Parameter')
plt.ylabel('Accuracy')
plt.ylim([0.6, 1])
plt.xscale('log')
plt.legend()

# AUC
plt.subplot(2, 1, 2, sharex=ax1)

plt.plot(alphas, auc_avg_raw, label='raw data')
plt.fill_between(alphas, [x+e for x, e in zip(auc_avg_raw, auc_err_raw)], [x-e for x, e in zip(auc_avg_raw, auc_err_raw)], alpha=0.5)

plt.plot(alphas, auc_avg_onehot, label='one-hot data')
plt.fill_between(alphas, [x+e for x, e in zip(auc_avg_onehot, auc_err_onehot)], [x-e for x, e in zip(auc_avg_onehot, auc_err_onehot)], alpha=0.5)

plt.xlabel('Regularization Parameter')
plt.ylabel('AUC')
plt.ylim([0.6, 1])
plt.xscale('log')
plt.legend()

plt.tight_layout()
plt.show()   


This tells us a couple of things. First, the raw data and the one-hot encoded data seem to perform approximately the same. However, it's important to keep in mind that L2 models use all of the features, so this may change when we introduce L1 regularization and the number of features decreases. Second, our error bars are fairly wide. Third, the best AUC we can hope for is about 0.95, and the best accuracy is about 0.85. Looking at the left side of the plots, very small amounts of regularization give you fairly good accuracy but low AUC. Looking at the right side of the plots, large amounts of regularization improve AUC, but decrease accuracy. The plots also give us a range of regularization parameters (alpha) to consider.

# Tune an Elastic Net Model
The challenge in tuning an elastic net model is that we have two parameters to consider:
    1. alpha: the amount of regularization 
    2. l1_ratio: the ratio between L1 and L2 regularization
We're interested in understanding how performance (measured by accuracy and AUC) vary with the number of parameters used. To do this, we'll start by fixing alpha, and then varying the l1_ratio to see a range of models with differing number of parameters.

In [ ]:
def tune_l1_ratio(X, k, alpha, l1_ratios):
    accuracy_avg = []
    accuracy_err = []
    auc_avg = []
    auc_err = []
    nzcoeffs_median = []

    for l1_ratio in l1_ratios:
        accuracy = np.zeros((k,))
        auroc = np.zeros((k,))
        nzcoeffs = np.zeros((k,))

        for i, (split_test_indices, split_test_videos) in enumerate(zip(*split_by_video(train_videos, k))):
            split_train_indices = list(set(train_indices) - set(split_test_indices))

            # fit model
            sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True)
            sgdc.fit(X[split_train_indices, :], y[split_train_indices])

            # accuracy
            accuracy[i] = sgdc.score(X[split_test_indices, :], y[split_test_indices])

            # auc
            yhat = sgdc.predict_proba(X[split_test_indices, :])[:, 1]
            fpr, tpr, thresholds = roc_curve(y[split_test_indices], yhat)
            auroc[i] = auc(fpr, tpr)
            
            # nonzero coeffs
            nzcoeffs[i] = np.sum(sgdc.coef_[0]!=0)

        # calculate mean and 95% confidence intervals
        accuracy_avg.append(np.mean(accuracy))
        ci = st.t.interval(0.95, k-1, loc=np.mean(accuracy), scale=st.sem(accuracy))
        accuracy_err.append((ci[1]-ci[0])/2)
        auc_avg.append(np.mean(auroc))
        ci = st.t.interval(0.95, k-1, loc=np.mean(auroc), scale=st.sem(auroc))
        auc_err.append((ci[1]-ci[0])/2)
        nzcoeffs_median.append(np.median(nzcoeffs))
    return accuracy_avg, accuracy_err, auc_avg, auc_err, nzcoeffs_median

k = 5
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
l1_ratios = [0.2, 0.4, 0.6, 0.8, 1.0]


fig, axes = plt.subplots(2, 2, sharex='col', sharey=True, figsize=(10, 10))
axes[0, 0].axhline(y=0.85, linestyle='--', color='black', alpha=0.5)
axes[0, 1].axhline(y=0.85, linestyle='--', color='black', alpha=0.5)
axes[1, 0].axhline(y=0.95, linestyle='--', color='black', alpha=0.5)
axes[1, 1].axhline(y=0.95, linestyle='--', color='black', alpha=0.5)

for alpha in alphas:
    accuracy_avg_raw, accuracy_err_raw, auc_avg_raw, auc_err_raw, nzcoeffs_median_raw = tune_l1_ratio(X, k, alpha, l1_ratios)
    accuracy_avg_onehot, accuracy_err_onehot, auc_avg_onehot, auc_err_onehot, nzcoeffs_median_onehot = tune_l1_ratio(X_one_hot, k, alpha, l1_ratios)

    # Accuracy
    axes[0, 0].plot(nzcoeffs_median_raw, accuracy_avg_raw, label='alpha %.0e' % alpha)
    #axes[0, 0].fill_between(nzcoeffs_median_raw, [x+e for x, e in zip(accuracy_avg_raw, accuracy_err_raw)], [x-e for x, e in zip(accuracy_avg_raw, accuracy_err_raw)], alpha=0.5)
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_ylim([0.6, 1])
    axes[0, 0].set_title('Raw data Accuracy')
    axes[0, 0].legend()
    

    axes[0, 1].plot(nzcoeffs_median_onehot, accuracy_avg_onehot, label='alpha %.0e' % alpha)
    #axes[0, 1].fill_between(nzcoeffs_median_onehot, [x+e for x, e in zip(accuracy_avg_onehot, accuracy_err_onehot)], [x-e for x, e in zip(accuracy_avg_onehot, accuracy_err_onehot)], alpha=0.5)
    axes[0, 1].set_ylim([0.6, 1])
    axes[0, 1].set_title('One-hot data Accuracy')
    axes[0, 1].legend()
    
    # AUC
    axes[1, 0].plot(nzcoeffs_median_raw, auc_avg_raw, label='alpha %.0e' % alpha)
    #axes[1, 0].fill_between(nzcoeffs_median_raw, [x+e for x, e in zip(auc_avg_raw, auc_err_raw)], [x-e for x, e in zip(auc_avg_raw, auc_err_raw)], alpha=0.5)
    axes[1, 0].set_xlabel('Features')
    axes[1, 0].set_ylabel('AUC')
    axes[1, 0].set_ylim([0.6, 1])
    axes[1, 0].set_title('Raw data AUC')
    axes[1, 0].legend()
    
    axes[1, 1].plot(nzcoeffs_median_onehot, auc_avg_onehot, label='alpha %.0e' % alpha)
    #axes[1, 1].fill_between(nzcoeffs_median_onehot, [x+e for x, e in zip(auc_avg_onehot, auc_err_onehot)], [x-e for x, e in zip(auc_avg_onehot, auc_err_onehot)], alpha=0.5)
    axes[1, 1].set_xlabel('Features')
    axes[1, 1].set_ylim([0.6, 1])
    axes[1, 1].set_title('One-hot AUC')
    axes[1, 1].legend()

plt.tight_layout()
plt.show()   

Looking at these plots, we can easily choose alpha for each dataset. For the raw data, we should use alpha=0.1. For the one-hot data, we should use alpha=0.01, unless we want a model with less than 20 or so features, in which case we should use alpha=0.1, but we need to be careful because accuracy and AUC drop off quickly.

# Better regularization for one-hot

In [ ]:
from cvxpy import *
from sklearn.metrics import accuracy_score

m, n = X_one_hot.shape

fused = np.zeros((n+1, n))
# define fused lasso matrix
index = 0
for i in range(30):
    start, end = q_to_start_index[i], q_to_start_index[i+1]
    fused[index, start] = 1
    index += 1
    for j in range(start, end-1):
        fused[index, j] = -1
        fused[index, j+1] = 1
        index += 1

# tune fused regularization
accuracy_avg = []
accuracy_err = []
auc_avg = []
auc_err = []
nzcoeffs_median = []

lambdas = np.logspace(-2, 1.5, 20)
for lamb in lambdas:
    accuracy = np.zeros((k,))
    auroc = np.zeros((k,))
    nzcoeffs = np.zeros((k,))

    for i, (split_test_indices, split_test_videos) in enumerate(zip(*split_by_video(train_videos, k))):
        split_train_indices = list(set(train_indices) - set(split_test_indices))

        # fit model
        beta = Variable(n)
        beta0 = Variable()
        objective = Minimize(sum_entries(logistic(mul_elemwise(-((2*y)-1)[split_train_indices], (X_one_hot[split_train_indices, :] * beta)+beta0))) + \
                             0.1*norm(beta, 2) + lamb*norm(fused * beta, 1))
        prob = Problem(objective, [])
        result = prob.solve()

        b = np.array(beta.value).reshape((n,))
        b[np.abs(b) < pow(10, -7)] = 0
        yhat = beta0.value+(X_one_hot[split_test_indices, :].dot(b))
        fpr, tpr, thresholds = roc_curve(y[split_test_indices], yhat)
        nnz = np.sum(~np.isclose(b[1:], b[0:-1]) & (b[0:-1]!=0))

        # accuracy
        accuracy[i] = accuracy_score(y[split_test_indices], yhat>0)

        # auc
        auroc[i] = auc(fpr, tpr)
            
        # nonzero coeffs
        nzcoeffs[i] = nnz

    # calculate mean and 95% confidence intervals
    accuracy_avg.append(np.mean(accuracy))
    ci = st.t.interval(0.95, k-1, loc=np.mean(accuracy), scale=st.sem(accuracy))
    accuracy_err.append((ci[1]-ci[0])/2)
    auc_avg.append(np.mean(auroc))
    ci = st.t.interval(0.95, k-1, loc=np.mean(auroc), scale=st.sem(auroc))
    auc_err.append((ci[1]-ci[0])/2)
    nzcoeffs_median.append(np.median(nzcoeffs))

plt.figure(figsize=(15, 6))
# Accuracy
plt.plot(nzcoeffs_median, accuracy_avg)
plt.fill_between(nzcoeffs_median, [x+e for x, e in zip(accuracy_avg, accuracy_err)], [x-e for x, e in zip(accuracy_avg, accuracy_err)], alpha=0.5, label='accuracy')

# AUC
plt.plot(nzcoeffs_median, auc_avg)
plt.fill_between(nzcoeffs_median, [x+e for x, e in zip(auc_avg, auc_err)], [x-e for x, e in zip(auc_avg, auc_err)], alpha=0.5, label='auc')

plt.xlabel('Features')
plt.legend()
plt.show()

In [ ]:
# Best L2
sgdc = SGDClassifier(loss="log", penalty='l2', alpha=0.01, fit_intercept=True)
sgdc.fit(X[train_indices, :], y[train_indices])
print_stats(sgdc, X[train_indices, :], y[train_indices], X[test_indices, :], y[test_indices])

yhat = sgdc.predict_proba(X[test_indices, :])[:, 1]
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
plt.plot(fpr, tpr, label='L2 (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best Raw data L1
sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=0.1, l1_ratio=0.9, fit_intercept=True)
sgdc.fit(X[train_indices, :], y[train_indices])
print_stats(sgdc, X[train_indices, :], y[train_indices], X[test_indices, :], y[test_indices])

yhat = sgdc.predict_proba(X[test_indices, :])[:, 1]
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
print('questions used', [header[i] for i, x in enumerate(sgdc.coef_[0]) if x != 0])
plt.plot(fpr, tpr, label='L1 raw data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best On-hot data L1
sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=0.01, l1_ratio=0.9, fit_intercept=True)
sgdc.fit(X_one_hot[train_indices, :], y[train_indices])
print_stats(sgdc, X_one_hot[train_indices, :], y[train_indices], X_one_hot[test_indices, :], y[test_indices])

yhat = sgdc.predict_proba(X_one_hot[test_indices, :])[:, 1]
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
print('questions used', [header_one_hot[i] for i, x in enumerate(sgdc.coef_[0]) if x != 0])
plt.plot(fpr, tpr, label='L1 one-hot data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best On-hot fused
beta = Variable(n)
beta0 = Variable()
objective = Minimize(sum_entries(logistic(mul_elemwise(-((2*y)-1)[train_indices], (X_one_hot[train_indices, :] * beta)+beta0))) + \
                             0.1*norm(beta, 2) + 0*norm(beta, 1) + 10*norm(fused * beta, 1))
prob = Problem(objective, [])
result = prob.solve()

b = np.array(beta.value).reshape((n,))
b[np.abs(b) < pow(10, -7)] = 0
nnz = np.sum(~np.isclose(b[1:], b[0:-1]) & (b[0:-1]!=0))
print('nonzero coefficients:', nnz, 'out of', X_one_hot.shape[1])
yhat = beta0.value+(X_one_hot[train_indices, :].dot(b))
fpr, tpr, thresholds = roc_curve(y[train_indices], yhat)
print('train accuracy:', accuracy_score(y[train_indices], yhat>0))
print('train auc:', auc(fpr, tpr))
yhat = beta0.value+(X_one_hot[test_indices, :].dot(b))
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
print('test accuracy:', accuracy_score(y[test_indices], yhat>0))
print('test auc:', auc(fpr, tpr))
print('questions used', [header_one_hot[i] for i, x in enumerate(b) if b[i] != 0 and not np.isclose(b[i], b[i-1])])
plt.plot(fpr, tpr, label='Fused one-hot data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

So our L1-regularized models are performing about as well as our L2 model, even though they're using fewer features. Also, even though the L1 one-hot data model is using more features, those features are much simpler (yes/no questions) than the features used in the L2 and L1 raw data model where each question has 6 responses. I chose parameters for these three models by looking at the cross-validation plots. Since the error bars on our cross-validation runs are fairly large, we really don't need to tune more precisely than this. Next, let's try to really crank up the regularization to produce L1 models with as few features as possible.

In [ ]:
# Best L2
sgdc = SGDClassifier(loss="log", penalty='l2', alpha=0.01, fit_intercept=True)
sgdc.fit(X[train_indices, :], y[train_indices])
print_stats(sgdc, X[train_indices, :], y[train_indices], X[test_indices, :], y[test_indices])

yhat = sgdc.predict_proba(X[test_indices, :])[:, 1]
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
plt.plot(fpr, tpr, label='L2 (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best Raw data L1
sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=0.1, l1_ratio=0.95, fit_intercept=True)
sgdc.fit(X[train_indices, :], y[train_indices])
print_stats(sgdc, X[train_indices, :], y[train_indices], X[test_indices, :], y[test_indices])

yhat = sgdc.predict_proba(X[test_indices, :])[:, 1]
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
print('questions used', [header[i] for i, x in enumerate(sgdc.coef_[0]) if x != 0])
plt.plot(fpr, tpr, label='L1 raw data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best One-hot data L1
sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=0.1, l1_ratio=0.9, fit_intercept=True)
sgdc.fit(X_one_hot[train_indices, :], y[train_indices])
print_stats(sgdc, X_one_hot[train_indices, :], y[train_indices], X_one_hot[test_indices, :], y[test_indices])

yhat = sgdc.predict_proba(X_one_hot[test_indices, :])[:, 1]
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
print('questions used', [header_one_hot[i] for i, x in enumerate(sgdc.coef_[0]) if x != 0])
plt.plot(fpr, tpr, label='L1 one-hot data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best On-hot fused
beta = Variable(n)
beta0 = Variable()
objective = Minimize(sum_entries(logistic(mul_elemwise(-((2*y[train_indices])-1), (X_one_hot[train_indices, :] * beta)+beta0))) + \
                             0.1*norm(beta, 2) + 0*norm(beta, 1) + 30*norm(fused * beta, 1))
prob = Problem(objective, [])
result = prob.solve()

b = np.array(beta.value).reshape((n,))
b[np.abs(b) < pow(10, -7)] = 0
nnz = np.sum(~np.isclose(b[1:], b[0:-1]) & (b[0:-1]!=0))
print('nonzero coefficients:', nnz, 'out of', X_one_hot.shape[1])
yhat = beta0.value+(X_one_hot[train_indices, :].dot(b))
fpr, tpr, thresholds = roc_curve(y[train_indices], yhat)
print('train accuracy:', accuracy_score(y[train_indices], yhat>0))
print('train auc:', auc(fpr, tpr))
yhat = beta0.value+(X_one_hot[test_indices, :].dot(b))
fpr, tpr, thresholds = roc_curve(y[test_indices], yhat)
print('test accuracy:', accuracy_score(y[test_indices], yhat>0))
print('test auc:', auc(fpr, tpr))
print('questions used', [header_one_hot[i] for i, x in enumerate(b) if b[i] != 0 and not np.isclose(b[i], b[i-1])])
plt.plot(fpr, tpr, label='Fused one-hot data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

We're using a small number of features here and the AUC's look pretty good, but the accuracies are low. This ties back to the very first plot - why is it that increased regularization corresponds to better AUC's but lower accuracy?

We can think of a classifier like this as a method for predicting whether or not the child in the video has autism. But we can also think of it as a method for ordering all of the children from most likely to have autism to least likely to have autism. In order to make good predictions, we need to correctly order the children and then correctly choose a cutoff so that we can say all the children with a score above the cutoff have autism and all of the others don't. We might get the ordering right, but if we get the cutoff wrong, our accuracy will be bad. An AUC curve tells you if your ordering is good. Accuracy tells you if your ordering AND your cutoff are good. 

A linear model like this will set the cutoff based on the prevalence of autism in the training data. Since we have a small dataset, the prevalence of autism in the training set might be slightly different than the prevalence of autism in the test set, just by chance. This will cause the model to choose a bad cutoff, but the ordering it produces might still be good. I think that's what's happening here. The problem will get worse if you try to run this model on a new dataset with a very different prevalence.

However, I think you can make the case that since the purpose of your method is to triage children so that clinicians can diagnose them more quickly, then maybe you don't need to worry about the cutoff. It may be that being able to order children from most likely to have autism to least likely is sufficient. In this case, you don't need to worry too much about accuracy and you can focus on finding a model with a good AUC.

In [ ]:
train_prevalence = np.sum(y[train_indices])
print('train prevalence', np.sum(y[train_indices]/len(train_indices)))
print('test prevalence', np.sum(y[test_indices]/len(test_indices)))
print(np.shape(train_indices), np.shape(test_indices))

If we take a look at train prevalence vs test prevalance, we see they they do differ by quite a bit.

# Independent Test Set

In [ ]:
# Best L2
sgdc = SGDClassifier(loss="log", penalty='l2', alpha=0.01, fit_intercept=True)
sgdc.fit(X, y)
print_stats(sgdc, X, y, Xind, yind)

yhat = sgdc.predict_proba(Xind)[:, 1]
fpr, tpr, thresholds = roc_curve(yind, yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
l2_beta = sgdc.coef_[0]
plt.plot(fpr, tpr, label='L2 (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best Raw data L1
sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=0.1, l1_ratio=0.9, fit_intercept=True)
sgdc.fit(X, y)
print_stats(sgdc, X, y, Xind, yind)

yhat = sgdc.predict_proba(Xind)[:, 1]
fpr, tpr, thresholds = roc_curve(yind, yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
l1_beta = sgdc.coef_[0]
print('questions used', [header[i] for i, x in enumerate(sgdc.coef_[0]) if x != 0])
plt.plot(fpr, tpr, label='L1 raw data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best On-hot data L1
sgdc = SGDClassifier(loss="log", penalty='elasticnet', alpha=0.01, l1_ratio=0.9, fit_intercept=True)
sgdc.fit(X_one_hot, y)
print_stats(sgdc, X_one_hot, y, Xind_one_hot, yind)

yhat = sgdc.predict_proba(Xind_one_hot)[:, 1]
fpr, tpr, thresholds = roc_curve(yind, yhat)
nnz = np.sum(sgdc.coef_[0]!=0)
l1_onehot_beta = sgdc.coef_[0]
print('questions used', [header_one_hot[i] for i, x in enumerate(sgdc.coef_[0]) if x != 0])
plt.plot(fpr, tpr, label='L1 one-hot data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

# Best One-hot fused
beta = Variable(X_one_hot.shape[1])
beta0 = Variable()
objective = Minimize(sum_entries(logistic(mul_elemwise(-((2*y)-1), (X_one_hot * beta)+beta0))) + \
                             0.1*norm(beta, 2) + 10*norm(fused * beta, 1))
prob = Problem(objective, [])
result = prob.solve()

b = np.array(beta.value).reshape((X_one_hot.shape[1],))
b[np.abs(b) < pow(10, -7)] = 0
nnz = np.sum(~np.isclose(b[1:], b[0:-1]) & (b[0:-1]!=0))
l1_fused_beta = b
print('nonzero coefficients:', nnz, 'out of', X_one_hot.shape[1])
yhat = beta0.value+(X_one_hot.dot(b))
fpr, tpr, thresholds = roc_curve(y, yhat)
print('train accuracy:', accuracy_score(y, yhat>0))
print('train auc:', auc(fpr, tpr))
yhat = beta0.value+(Xind_one_hot.dot(b))
fpr, tpr, thresholds = roc_curve(yind, yhat)
print('test accuracy:', accuracy_score(yind, yhat>0))
print('test auc:', auc(fpr, tpr))
print('questions used', [header_one_hot[i] for i, x in enumerate(b) if b[i] != 0 and not np.isclose(b[i], b[i-1])])
plt.plot(fpr, tpr, label='Fused one-hot data (features=%d, auc=%f)' % (nnz, auc(fpr, tpr)))

plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
print(l1_onehot_beta)
print(l1_fused_beta)

In [ ]:
# Compare feature functions
plt.figure(figsize=(20, 20))
for q in range(30):
    plt.subplot(6, 5, q+1)
    responses = [x[1] for x in header_one_hot if x[0] == 'question%d' % (q+1) and x[1] < 7]
    plt.plot(responses, [l2_beta[q]*r for r in responses], label='l2')
    plt.plot(responses, [l1_beta[q]*r for r in responses], label='l1')
    plt.plot(responses, [l1_onehot_beta[i] for i in range(q_to_start_index[q], q_to_start_index[q]+len(responses))], label='l1_onehot')
    plt.plot(responses, [l1_fused_beta[i] for i in range(q_to_start_index[q], q_to_start_index[q]+len(responses))], label='fused_onehot')
    plt.ylim([-1, 1])
    plt.title('Question %d' % (q+1))
    plt.legend()
plt.show()

# Look at concordance

In [ ]:
unique_scorers = sorted(set([x[0] for x in metadata]))
unique_videos = sorted(set([x[1] for x in metadata]))
scorer_to_index = dict([(x, i) for i, x in enumerate(unique_scorers)])
video_to_index = dict([(x, i) for i, x in enumerate(unique_videos)])

X_conc = np.zeros((len(unique_scorers), len(unique_videos), X.shape[1]))-1
for i, m in enumerate(metadata):
    X_conc[scorer_to_index[m[0]], video_to_index[m[1]], :] = X[i, :]

Concordance between raters:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

q = 10
plt.figure(figsize=(15, 5))
plt.subplot(2, 3, 1)
cm = confusion_matrix(X_conc[0, :, q], X_conc[1, :, q])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('0 vs 1')
plt.colorbar()

plt.subplot(2, 3, 2)
cm = confusion_matrix(X_conc[1, :, q], X_conc[2, :, q])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('1 vs 2')
plt.colorbar()

plt.subplot(2, 3, 3)
cm = confusion_matrix(X_conc[2, :, q], X_conc[1, :, q])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('2 vs 1')
plt.colorbar()

plt.subplot(2, 3, 4)
cm = confusion_matrix(X_conc[0, :, q], X_conc[2, :, q])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('0 vs 2')
plt.colorbar()

plt.subplot(2, 3, 5)
cm = confusion_matrix(X_conc[1, :, q], X_conc[0, :, q])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('1 vs 0')
plt.colorbar()

plt.subplot(2, 3, 6)
cm = confusion_matrix(X_conc[2, :, q], X_conc[0, :, q])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('2 vs 0')
plt.colorbar()

plt.tight_layout()
plt.show()

Concordance between videos:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

v = 10
plt.figure(figsize=(15, 5))
plt.subplot(2, 3, 1)
cm = confusion_matrix(X_conc[0, v, :], X_conc[1, v, :])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('0 vs 1')
plt.colorbar()

plt.subplot(2, 3, 2)
cm = confusion_matrix(X_conc[1, v, :], X_conc[2, v, :])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('1 vs 2')
plt.colorbar()

plt.subplot(2, 3, 3)
cm = confusion_matrix(X_conc[2, v, :], X_conc[1, v, :])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('2 vs 1')
plt.colorbar()

plt.subplot(2, 3, 4)
cm = confusion_matrix(X_conc[0, v, :], X_conc[2, v, :])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('0 vs 2')
plt.colorbar()

plt.subplot(2, 3, 5)
cm = confusion_matrix(X_conc[1, v, :], X_conc[0, v, :])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('1 vs 0')
plt.colorbar()

plt.subplot(2, 3, 6)
cm = confusion_matrix(X_conc[2, v, :], X_conc[0, v, :])
plt.imshow(cm/np.sum(cm, axis=0), vmin=0, vmax=1)
plt.title('2 vs 0')
plt.colorbar()

plt.tight_layout()
plt.show()